Appunti:Prendo le def
calcolo i termini più frequenti
faccio disambiguation
ottengo synset piu frequenti - che saranno i possibili genus
confronto
Il punto è non cofrontare tutte le definizioni (100K+) ma confrontarle solo con i genus

In [6]:
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from collections import Counter
from nltk.wsd import lesk

## Utilities

In [15]:
def word_sense_disambiguation(list_words, word):
    right_synset = lesk(list_words, word)
    return right_synset

print(word_sense_disambiguation(['i', 'saw', 'concert', 'live', 'in', 'the,' 'stadium', 'sing'], 'live').definition())

def remove_stop_words(row):
    stop_words = set(stopwords.words('english'))
    punctuation = [',', '.', ';', '!', '?', "'", "''", '"', "’", "’’", "`","``"]
    filtered_sentence = [w for w in row if not w.lower() in stop_words and w not in punctuation]
    return filtered_sentence

def calculate_frequency(row):
    freq_dict = {}
    row = remove_stop_words(word_tokenize(row))
    for word in row[1:]:
        if word.lower() not in freq_dict:
            freq_dict[word.lower()] = 1
        else:
            freq_dict[word.lower()] += 1
    return freq_dict

lead a certain kind of life; live in a certain style


## Execution

In [8]:

with open ('../def.csv', 'r') as f:
    for row in f:
        dict = calculate_frequency(row)
        k = Counter(dict)
        most_freq = k.most_common(3)
        
        print(most_freq)
        for word in most_freq:
            print(f"Word: {word} \n{wn.synsets(word[0])[0].hyponyms()}")
        
        print("\n\n\n")
        #synset.lexname()
        

[('feeling', 10), ('human', 8), ('feel', 8)]
Word: ('feeling', 10) 
[Synset('affect.n.01'), Synset('affection.n.01'), Synset('agitation.n.03'), Synset('ambivalence.n.01'), Synset('apathy.n.01'), Synset('astonishment.n.01'), Synset('calmness.n.03'), Synset('complex.n.03'), Synset('desire.n.01'), Synset('despair.n.02'), Synset('devastation.n.02'), Synset('dislike.n.02'), Synset('emotion.n.01'), Synset('enthusiasm.n.01'), Synset('expectation.n.03'), Synset('faintness.n.01'), Synset('fearlessness.n.01'), Synset('glow.n.04'), Synset('gratitude.n.01'), Synset('gravity.n.03'), Synset('happiness.n.02'), Synset('hope.n.02'), Synset('humility.n.02'), Synset('ingratitude.n.01'), Synset('levity.n.01'), Synset('liking.n.01'), Synset('pain.n.02'), Synset('pang.n.01'), Synset('passion.n.01'), Synset('pleasure.n.01'), Synset('pride.n.01'), Synset('sadness.n.01'), Synset('sensitivity.n.03'), Synset('sentiment.n.01'), Synset('sex.n.03'), Synset('shame.n.01'), Synset('soul.n.03'), Synset('sympathy.n.02')